In [51]:
import glob
import pandas as pd
import json
import csv



path = r'annotations'  # use your path
allFiles = glob.glob(path + "/*.csv")

data_samples1 = []
data_number1 = []
data_label1 = []

cnt = 0

for file in allFiles:
    allFiles = allFiles[:85]
    dictCollection = {"Action First-Party":[],
                      "Purpose":[],
                      "Personal Information Type":[]
                      }

    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    # df_tail = df.tail(1)[4]
    # print(df_tail)
    number_of_segments = 5#len(df) + 1

    file = file.split('annotations/', 1)[1]
    print(file)
    for i in range(number_of_segments-1):
        # print(i)
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Action First-Party"]["selectedText"])
                data_number1.append(1)
                data_label1.append("Action First-Party")
                cnt = cnt+1
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                data_number1.append(2)
                data_label1.append("Personal Information Type")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Purpose"]["selectedText"])
                data_number1.append(3)
                data_label1.append("Purpose")     
                cnt = cnt+1
        if df[5][i] == "Third Party Sharing/Collection":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                data_number1.append(2)
                data_label1.append("Personal Information Type")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Purpose"]["selectedText"])
                data_number1.append(3)
                data_label1.append("Purpose")     
                cnt = cnt+1
        if df[5][i] == "User Choice/Control":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                data_number1.append(2)
                data_label1.append("Personal Information Type")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Purpose"]["selectedText"])
                data_number1.append(3)
                data_label1.append("Purpose")     
                cnt = cnt+1
        

1017_sci-news.com.csv
{'Collection Mode': {'selectedText': 'nformation that you provide by filling in forms on our site', 'startIndexInSegment': 139, 'endIndexInSegment': 198, 'value': 'Explicit'}, 'Choice Scope': {'selectedText': 'null', 'startIndexInSegment': -1, 'endIndexInSegment': -1, 'value': 'not-selected'}, 'Action First-Party': {'selectedText': 'Sci-News.com may collect and process', 'startIndexInSegment': 59, 'endIndexInSegment': 95, 'value': 'Unspecified'}, 'Personal Information Type': {'selectedText': 'including', 'startIndexInSegment': 200, 'endIndexInSegment': 209, 'value': 'Unspecified'}, 'Choice Type': {'selectedText': 'Not selected', 'startIndexInSegment': -1, 'endIndexInSegment': -1, 'value': 'Unspecified'}, 'Identifiability': {'selectedText': 'null', 'startIndexInSegment': -1, 'endIndexInSegment': -1, 'value': 'not-selected'}, 'Does/Does Not': {'selectedText': 'null', 'startIndexInSegment': -1, 'endIndexInSegment': -1, 'value': 'Does'}, 'User Type': {'selectedText': 

In [52]:
print(data_label1[:5])
print(len(data_label1))

['Action First-Party', 'Personal Information Type', 'Purpose', 'Action First-Party', 'Personal Information Type']
204


In [53]:
print(data_number1[:5])
print(len(data_number1))

[1, 2, 3, 1, 2]
204


In [54]:
import numpy as np
print(data_samples1[:5])
print(len(data_samples1))

train_samples = np.array(data_samples1[:160])
train_label = np.array(data_label1[:160])
train_number = np.array(data_number1[:160])

test_samples = np.array(data_samples1[161:204])
test_label = np.array(data_label1[161:204])
test_number = np.array(data_number1[161:204])
print(train_samples)

['Sci-News.com may collect and process', 'including', 'other purposes, for example when you report a problem with our site', 'Sci-News.com may collect and process', 'names, e-mail and website addresses']
204
['Sci-News.com may collect and process' 'including'
 'other purposes, for example when you report a problem with our site'
 'Sci-News.com may collect and process'
 'names, e-mail and website addresses'
 'other purposes, for example when you report a problem with our site'
 'collects information from our users at several different points on our Web site.'
 'information' 'we collect and receive' 'information'
 "We're always working to make our services even better. One of the ways we do that is by analyzing information"
 'information that you choose to publicly post to or display on a Site.'
 'to information that you choose to publicly post to or display on a Site'
 'This privacy policy does not apply to information that you choose to publicly post to or display on a Site.'
 'when yo

In [55]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts = count_vect.fit_transform(train_samples)
X_train_counts
count_vect.get_feature_names()
count_vect.get_stop_words()

In [56]:
count_vect.vocabulary_.get('address')

4

In [58]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(160, 173)

In [59]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(160, 173)

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(train_samples, train_number)

In [64]:
import numpy as np
predicted = text_clf.predict(test_samples)
np.mean(predicted == test_number)  

0.7441860465116279

In [65]:
from sklearn import metrics
print(metrics.classification_report(test_number, predicted,
                                        target_names=test_label))

metrics.confusion_matrix(test_number, predicted)

                           precision    recall  f1-score   support

                  Purpose       0.67      0.86      0.75         7
Personal Information Type       0.76      0.92      0.83        24
                  Purpose       0.80      0.33      0.47        12

              avg / total       0.76      0.74      0.72        43



array([[ 6,  1,  0],
       [ 1, 22,  1],
       [ 2,  6,  4]])

In [14]:
from sklearn.linear_model import SGDClassifier
text_clf1 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf1.fit(train_samples, train_number)
predicted1 = text_clf1.predict(test_samples)
np.mean(predicted1 == test_number)  

0.7441860465116279

In [15]:
from sklearn import metrics
print(metrics.classification_report(test_number, predicted,
                                        target_names=test_label))

metrics.confusion_matrix(test_number, predicted)


                           precision    recall  f1-score   support

                  Purpose       0.67      0.86      0.75         7
Personal Information Type       0.76      0.92      0.83        24
                  Purpose       0.80      0.33      0.47        12

              avg / total       0.76      0.74      0.72        43



array([[ 6,  1,  0],
       [ 1, 22,  1],
       [ 2,  6,  4]])

In [66]:
print(data_samples1[:5])
print(len(data_samples1))

train_samples1 = np.array(data_samples1[:190])
train_label1 = np.array(data_label1[:190])
train_number1 = np.array(data_number1[:190])

test_samples1 = np.array(data_samples1[191:204])
test_label1 = np.array(data_label1[191:204])
test_number1 = np.array(data_number1[191:204])
print(len(train_samples1))

['Sci-News.com may collect and process', 'including', 'other purposes, for example when you report a problem with our site', 'Sci-News.com may collect and process', 'names, e-mail and website addresses']
204
190


In [67]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect1 = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts1 = count_vect1.fit_transform(train_samples1)
X_train_counts1
print(count_vect1.get_feature_names())
count_vect1.get_stop_words()

['about', 'access', 'account', 'activities', 'additional', 'address', 'addresses', 'administer', 'ads', 'advertisements', 'advertisers', 'all', 'an', 'and', 'answer', 'any', 'apply', 'are', 'as', 'asked', 'at', 'automatically', 'be', 'bing', 'birthday', 'boards', 'browser', 'by', 'carry', 'certain', 'choose', 'code', 'collect', 'collected', 'collects', 'com', 'contact', 'content', 'contest', 'contests', 'cookies', 'creating', 'dart', 'design', 'details', 'determine', 'digital', 'directly', 'display', 'do', 'does', 'each', 'easier', 'email', 'entering', 'event', 'example', 'experience', 'features', 'files', 'for', 'from', 'give', 'google', 'have', 'however', 'identifiable', 'if', 'in', 'including', 'information', 'initiatives', 'interact', 'interactive', 'interesting', 'internet', 'ip', 'is', 'it', 'job', 'joining', 'limited', 'lists', 'loyalty', 'magazine', 'magazines', 'mail', 'mailing', 'make', 'may', 'meet', 'message', 'more', 'name', 'names', 'news', 'non', 'not', 'number', 'of', '

In [68]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer1 = TfidfTransformer(use_idf=False).fit(X_train_counts1)
X_train_tf1 = tf_transformer1.transform(X_train_counts1)
X_train_tf1.shape

(190, 191)

In [69]:
tfidf_transformer1 = TfidfTransformer()
X_train_tfidf1 = tfidf_transformer1.fit_transform(X_train_counts1)
X_train_tfidf1.shape

(190, 191)

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf1 = text_clf1.fit(train_samples1, train_number1)

In [78]:
import numpy as np
predicted = text_clf1.predict(test_samples1)
np.mean(predicted == test_number1)  

0.76923076923076927

In [79]:
from sklearn import metrics
print(metrics.classification_report(test_number1, predicted,
                                        target_names=test_label1))

metrics.confusion_matrix(test_number1, predicted)

                           precision    recall  f1-score   support

       Action First-Party       0.33      1.00      0.50         1
Personal Information Type       0.86      0.86      0.86         7
                  Purpose       1.00      0.60      0.75         5

              avg / total       0.87      0.77      0.79        13



array([[1, 0, 0],
       [1, 6, 0],
       [1, 1, 3]])

In [75]:
from sklearn.linear_model import SGDClassifier
text_clf1 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf1.fit(train_samples1, train_number1)
predicted = text_clf1.predict(test_samples1)
np.mean(predicted == test_number1)  

0.76923076923076927

In [76]:
from sklearn import metrics
print(metrics.classification_report(test_number1, predicted,
                                        target_names=test_label1))

metrics.confusion_matrix(test_number1, predicted)

                           precision    recall  f1-score   support

       Action First-Party       0.50      1.00      0.67         1
Personal Information Type       0.83      0.71      0.77         7
                  Purpose       0.80      0.80      0.80         5

              avg / total       0.79      0.77      0.77        13



array([[1, 0, 0],
       [1, 5, 1],
       [0, 1, 4]])

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect1 = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts1 = count_vect1.fit_transform(data_samples1)
X_train_counts1

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer1 = TfidfTransformer(use_idf=False).fit(X_train_counts1)
X_train_tf1 = tf_transformer1.transform(X_train_counts1)
X_train_tf1.shape

tfidf_transformer1 = TfidfTransformer()
X_train_tfidf1 = tfidf_transformer1.fit_transform(X_train_counts1)
X_train_tfidf1.shape

(204, 205)

In [89]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf1 = text_clf1.fit(data_samples1, data_number1)

import numpy as np
predicted = text_clf1.predict(data_samples1)
np.mean(predicted == data_number1)  

0.93627450980392157

In [84]:
from sklearn import metrics
print(metrics.classification_report(data_number1, predicted,
                                        target_names=data_label1))

metrics.confusion_matrix(data_number1, predicted)

                           precision    recall  f1-score   support

       Action First-Party       0.88      0.95      0.91        60
Personal Information Type       0.95      0.97      0.96        90
                  Purpose       1.00      0.87      0.93        54

              avg / total       0.94      0.94      0.94       204



array([[57,  3,  0],
       [ 3, 87,  0],
       [ 5,  2, 47]])

In [85]:
from sklearn.linear_model import SGDClassifier
text_clf1 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf1.fit(data_samples1, data_number1)

import numpy as np
predicted = text_clf1.predict(data_samples1)
np.mean(predicted == data_number1)  

0.99509803921568629

In [86]:
from sklearn import metrics
print(metrics.classification_report(data_number1, predicted,
                                        target_names=data_label1))

metrics.confusion_matrix(data_number1, predicted)

                           precision    recall  f1-score   support

       Action First-Party       1.00      0.98      0.99        60
Personal Information Type       0.99      1.00      0.99        90
                  Purpose       1.00      1.00      1.00        54

              avg / total       1.00      1.00      1.00       204



array([[59,  1,  0],
       [ 0, 90,  0],
       [ 0,  0, 54]])

In [90]:

path = r'annotations'  # use your path
allFiles = glob.glob(path + "/*.csv")

data_samples2 = []
data_number2 = []
data_label2 = []

cnt = 0

for file in allFiles:
    allFiles = allFiles[:85]
    dictCollection = {"Action First-Party":[],
                      "Purpose":[],
                      "Personal Information Type":[]
                      }

    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    # df_tail = df.tail(1)[4]
    # print(df_tail)
    number_of_segments = 5#len(df) + 1

    file = file.split('annotations/', 1)[1]
    print(file)
    for i in range(number_of_segments-1):
        # print(i)
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Action First-Party"]["selectedText"])
                data_number2.append(1)
                data_label2.append("Action First-Party")
                cnt = cnt+1
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Personal Information Type"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Purpose"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")     
                cnt = cnt+1
        if df[5][i] == "Third Party Sharing/Collection":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Personal Information Type"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Purpose"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")     
                cnt = cnt+1
        if df[5][i] == "User Choice/Control":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Personal Information Type"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples2.append(parse_json["Purpose"]["selectedText"])
                data_number2.append(2)
                data_label2.append("Other")     
                cnt = cnt+1
        

1017_sci-news.com.csv
1028_redorbit.com.csv
1034_aol.com.csv
1050_honda.com.csv
105_amazon.com.csv
1070_wnep.com.csv
1083_highgearmedia.com.csv
1089_freep.com.csv
1099_enthusiastnetwork.com.csv
1106_allstate.com.csv
1164_acbj.com.csv
1205_opensecrets.org.csv
{'Third Party Entity': {'endIndexInSegment': 89, 'startIndexInSegment': 76, 'selectedText': 'third parties', 'value': 'Unnamed third party'}, 'Choice Scope': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Purpose': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Personal Information Type': {'endIndexInSegment': 68, 'startIndexInSegment': 33, 'selectedText': 'personally identifiable information', 'value': 'Generic personal information'}, 'Choice Type': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Identifiability': {'endIndexInSegment': 68, 'startIndexInSegment': 33, 'selectedText': 'personally identifiable information', 'value': 'Identif

In [91]:

train_samples2 = np.array(data_samples2[:160])
train_label2 = np.array(data_label2[:160])
train_number2 = np.array(data_number2[:160])

test_samples2 = np.array(data_samples2[161:204])
test_label2 = np.array(data_label2[161:204])
test_number2 = np.array(data_number2[161:204])


In [92]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect1 = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts1 = count_vect1.fit_transform(train_samples2)
X_train_counts1

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer1 = TfidfTransformer(use_idf=False).fit(X_train_counts1)
X_train_tf1 = tf_transformer1.transform(X_train_counts1)
X_train_tf1.shape

tfidf_transformer1 = TfidfTransformer()
X_train_tfidf1 = tfidf_transformer1.fit_transform(X_train_counts1)
X_train_tfidf1.shape

(160, 173)

In [93]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf1 = text_clf1.fit(train_samples2, train_number2)

import numpy as np
predicted = text_clf1.predict(test_samples2)
np.mean(predicted == test_number2)

0.90697674418604646

In [96]:
from sklearn import metrics
print(metrics.classification_report(test_number2, predicted,
                                        target_names=test_label2))

metrics.confusion_matrix(test_number2, predicted)
print(test_label2)

                    precision    recall  f1-score   support

             Other       1.00      0.43      0.60         7
             Other       0.90      1.00      0.95        36

       avg / total       0.92      0.91      0.89        43

['Other' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Action First-Party' 'Other' 'Action First-Party' 'Other' 'Other' 'Other'
 'Action First-Party' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Action First-Party' 'Other' 'Other'
 'Action First-Party' 'Other' 'Action First-Party' 'Other'
 'Action First-Party' 'Other' 'Other' 'Other' 'Other' 'Other' 'Other'
 'Other' 'Other' 'Other' 'Other' 'Other' 'Other']


In [50]:
from sklearn.linear_model import SGDClassifier
text_clf1 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf1.fit(train_samples2, train_number2)

import numpy as np
predicted = text_clf1.predict(test_samples2)
np.mean(predicted == test_number2)  

0.93023255813953487

In [41]:

path = r'annotations'  # use your path
allFiles = glob.glob(path + "/*.csv")

data_samples3 = []
data_number3 = []
data_label3 = []

cnt = 0

for file in allFiles:
    allFiles = allFiles[:85]
    dictCollection = {"Action First-Party":[],
                      "Purpose":[],
                      "Personal Information Type":[]
                      }

    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    # df_tail = df.tail(1)[4]
    # print(df_tail)
    number_of_segments = 5#len(df) + 1

    file = file.split('annotations/', 1)[1]
    print(file)
    for i in range(number_of_segments-1):
        # print(i)
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                data_samples3.append(parse_json["Action First-Party"]["selectedText"])
                data_number3.append(1)
                data_label3.append("Other")
                cnt = cnt+1
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples3.append(parse_json["Personal Information Type"]["selectedText"])
                data_number3.append(1)
                data_label3.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples3.append(parse_json["Purpose"]["selectedText"])
                data_number3.append(2)
                data_label3.append("Purpose")     
                cnt = cnt+1
        if df[5][i] == "Third Party Sharing/Collection":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                data_number1.append(1)
                data_label1.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Purpose"]["selectedText"])
                data_number1.append(2)
                data_label1.append("Purpose")     
                cnt = cnt+1
        if df[5][i] == "User Choice/Control":
            parse_json = json.loads(str(df[6][i]))
            print(parse_json)
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Personal Information Type"]["selectedText"])
                data_number1.append(1)
                data_label1.append("Other")                
                cnt = cnt+1
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                data_samples1.append(parse_json["Purpose"]["selectedText"])
                data_number1.append(2)
                data_label1.append("Purpose")     
                cnt = cnt+1
        

1017_sci-news.com.csv
1028_redorbit.com.csv
1034_aol.com.csv
1050_honda.com.csv
105_amazon.com.csv
1070_wnep.com.csv
1083_highgearmedia.com.csv
1089_freep.com.csv
1099_enthusiastnetwork.com.csv
1106_allstate.com.csv
1164_acbj.com.csv
1205_opensecrets.org.csv
{'Third Party Entity': {'endIndexInSegment': 89, 'startIndexInSegment': 76, 'selectedText': 'third parties', 'value': 'Unnamed third party'}, 'Choice Scope': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Purpose': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Personal Information Type': {'endIndexInSegment': 68, 'startIndexInSegment': 33, 'selectedText': 'personally identifiable information', 'value': 'Generic personal information'}, 'Choice Type': {'endIndexInSegment': -1, 'startIndexInSegment': -1, 'value': 'Unspecified'}, 'Identifiability': {'endIndexInSegment': 68, 'startIndexInSegment': 33, 'selectedText': 'personally identifiable information', 'value': 'Identif

In [42]:
train_samples3 = np.array(data_samples3[:160])
train_label3 = np.array(data_label3[:160])
train_number3 = np.array(data_number3[:160])

test_samples3 = np.array(data_samples3[161:204])
test_label3 = np.array(data_label3[161:204])
test_number3 = np.array(data_number3[161:204])
print(train_number3)

[1 1 2 1 1 2 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 2 1 1 1 1 1 1 2
 1 1 2 1 1 1 1 2 1 1 1 2 1 1 2 1 1 2 1 1 1 1 1 1 2 1 1 2 1 1 1 2 1 1 1 1 1
 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 1 1 2 1 1 2 1 1 2 1 1 2
 1 1 1 1 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2 1 1 2
 1 2 1 1 1 1 1 1 1 1 1 1]


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect1 = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=1000)
X_train_counts1 = count_vect1.fit_transform(train_samples3)
X_train_counts1

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer1 = TfidfTransformer(use_idf=False).fit(X_train_counts1)
X_train_tf1 = tf_transformer1.transform(X_train_counts1)
X_train_tf1.shape

tfidf_transformer1 = TfidfTransformer()
X_train_tfidf1 = tfidf_transformer1.fit_transform(X_train_counts1)
X_train_tfidf1.shape

(160, 166)

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf1 = text_clf1.fit(train_samples3, train_number3)

import numpy as np
predicted = text_clf1.predict(test_samples3)
np.mean(predicted == test_number3)

0.83333333333333337

In [45]:
from sklearn.linear_model import SGDClassifier
text_clf1 = Pipeline([('vect', CountVectorizer()), 
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42))])
_ = text_clf1.fit(train_samples3, train_number3)

import numpy as np
predicted = text_clf1.predict(test_samples3)
np.mean(predicted == test_number3)  

0.66666666666666663